In [1]:
#Load final training df as generated from notebook 1.4
import pandas as pd

df = pd.read_csv("toy_dset.csv") #assume df within repository and not in local data repositories

#todo name cols accordingly
df.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,0.111525,0.003693,2.924585,0.196323,0.185644,0.957654,0.082177,0.194132,-40.034338,1.442674,...,0.007683,0.009630,0.012317,0.004561,0.007717,0.010147,0.014572,0.009849,0.032417,healthy
1,0.134430,0.009462,2.970514,0.243756,0.199684,0.965706,0.026270,0.269125,-55.721892,0.616560,...,0.006095,0.003532,0.008306,0.001782,0.002502,0.004635,0.006472,0.003845,0.036021,healthy
2,0.359400,0.007149,3.267919,0.421726,0.287201,2.886614,0.004029,0.745347,-25.974753,0.669882,...,0.014414,0.009584,0.018643,0.003762,0.006724,0.011085,0.015089,0.006036,0.009797,healthy
3,0.248244,0.000748,3.081517,0.439152,0.305792,1.678805,0.006238,0.615579,-42.873513,-0.027541,...,0.002670,0.001988,0.022535,0.001271,0.001595,0.002469,0.003605,0.002079,0.037430,healthy
4,0.223997,0.013682,2.871083,0.287654,0.178710,1.236823,0.116181,0.368188,-47.392346,0.737621,...,0.008279,0.006869,0.010639,0.001476,0.005267,0.008221,0.009075,0.005956,0.025121,healthy


In [4]:
#convert df to input output X Y 
from sklearn.model_selection import train_test_split 
from sklearn import svm

X = df.iloc[: , :-1] #everything except last column
y = df.iloc[:, 34] #last column
print(" shape", X.shape)
print("y shape", y.shape)


 shape (3945, 34)
y shape (3945,)


In [13]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#normalization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#apply pca to reduce feature space 
pca = PCA(n_components=0.95) #0.95 variance
X_pca = pca.fit_transform(X_scaled)

print(f"Number of components chosen: {pca.n_components_}")

Number of components chosen: 23


In [ ]:
#TODO more sophisticated train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Cross-validation scores: [0.38403042 0.40050697 0.39923954 0.38910013 0.37008872]
Mean Accuracy: 0.39 (+/- 0.02)


In [ ]:
#create the svm of choice

#simple svm
#clf = svm.SVC() #TODO test different kernels different params +

#polynomial svm
clf = svm.SVC()

#fit the svm to the data
clf.fit(X_train, y_train)


Best Parameters: {'svm__C': 1, 'svm__degree': 3, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}


In [11]:
#TODO accuracy metrics f1 ++
y_predict = clf.predict(X_test)

In [12]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix


print("\nClassification Report:")
print(classification_report(y_test, y_predict))


print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_predict))


Classification Report:
              precision    recall  f1-score   support

    COVID-19       0.37      0.47      0.41       241
     healthy       0.41      0.28      0.34       282
 symptomatic       0.38      0.41      0.39       266

    accuracy                           0.38       789
   macro avg       0.39      0.39      0.38       789
weighted avg       0.39      0.38      0.38       789


Confusion Matrix:
[[113  46  82]
 [107  80  95]
 [ 88  69 109]]
